# Анализ влияния товарного ассортимента на выручку ТТ ритейла

__Задача__

Исследовать влияние товарного ассортимента на выручку магазина в целях улучшения алгоритма подбора товарного ассортимента.

### Подготовка данных

In [1]:
import pandas as pd
#import folium
import warnings
warnings.filterwarnings('ignore')

Файл информации о ТТ:

In [9]:
zao = pd.read_csv('zao_info.csv', sep=';', usecols=[0, 5])

In [10]:
info_tt = pd.read_csv('zao_info.csv', sep=';')

Файлы чеков за апрель-май-июнь 2021 и апрель-май-июнь 2022

In [11]:
april_may21_chek = pd.read_csv('ch_zao_april-may21.csv', sep=';', usecols=[1, 4, 5], decimal=',')
june_jule21_chek = pd.read_csv('ch_zao_june-july21.csv', sep=';', usecols=[1, 4, 5], decimal=',')
april22_chek = pd.read_csv('ch_zao_april.csv', sep=';', usecols=[1, 4, 5], decimal=',')
may22_chek = pd.read_csv('ch_zao_may.csv', sep=';', usecols=[1, 4, 5], decimal=',')
june22_chek = pd.read_csv('ch_zao_june.csv', sep=';', usecols=[1, 4, 5], decimal=',')

In [12]:
april_may21_chek['CloseDate'] = pd.to_datetime(april_may21_chek['CloseDate'], format='%Y %m %d %H:%M:%S.%f').dt.round('S').dt.date
june_jule21_chek['CloseDate'] = pd.to_datetime(june_jule21_chek['CloseDate'], format='%Y %m %d %H:%M:%S.%f').dt.round('S').dt.date
april22_chek['CloseDate'] = pd.to_datetime(april22_chek['CloseDate'], format='%Y %m %d %H:%M:%S.%f').dt.round('S').dt.date
may22_chek['CloseDate'] = pd.to_datetime(may22_chek['CloseDate'], format='%Y %m %d %H:%M:%S.%f').dt.round('S').dt.date
june22_chek['CloseDate'] = pd.to_datetime(june22_chek['CloseDate'], format='%Y %m %d %H:%M:%S.%f').dt.round('S').dt.date

In [13]:
april_may21_chek = april_may21_chek.rename(columns={'id_TT': 'id_tt', 'CloseDate': 'date_ch'})
june_jule21_chek = june_jule21_chek.rename(columns={'id_TT': 'id_tt', 'CloseDate': 'date_ch'})
april22_chek = april22_chek.rename(columns={'id_TT': 'id_tt', 'CloseDate': 'date_ch'})
may22_chek= may22_chek.rename(columns={'id_TT': 'id_tt', 'CloseDate': 'date_ch'})
june22_chek= june22_chek.rename(columns={'id_TT': 'id_tt', 'CloseDate': 'date_ch'})

Файлы продаж за май 2021 и 2022 годов:

In [14]:
may21 = pd.read_csv('zao_mayl21.csv', sep=';', usecols=[1, 5, 6, 7, 8])
may22 = pd.read_csv('1_may.csv', sep=';', usecols=[0, 1, 2, 6, 7])

In [15]:
may21= may21.rename(columns={'id_tt_cl': 'id_tt', 'id_tov_cl': 'id_tov'})
may22= may22.rename(columns={'id_tt_cl': 'id_tt', 'id_tov_cl': 'id_tov'})

In [16]:
may21 = may21[['CheckUID', 'id_tt', 'id_tov', 'BaseSum', 'date_ch']]

Файл SKU:

In [17]:
sku = pd.read_csv('tov_info.csv', sep=';', usecols=[0, 1, 4, 5, 6, 7])

In [18]:
assort = sku[['id_tov', 'Name_tov', 'id_group', 'Name_gr', 'id_group2', 'Name_gr2']]

## Список ТТ для исследования:

Объединю таблицы:

In [19]:
chek = pd.concat([april_may21_chek, june_jule21_chek, april22_chek, may22_chek, june22_chek], ignore_index=True)

Изменю тип данных:

In [20]:
chek['date_ch'] = pd.to_datetime(chek['date_ch'])

In [21]:
chek['BaseSum'] = chek['BaseSum'].astype(int)

Вычислю помесячную выручку для каждого магазина

In [22]:
revenue = chek.groupby(['id_tt', pd.Grouper(key='date_ch', freq='M')])['BaseSum'].sum().reset_index()

Создам таблицу помесячной выручки:

In [23]:
revenue_pivot = pd.pivot_table(revenue,
               index=['id_tt'],
               columns=['date_ch'],
               values=['BaseSum'],
               )

In [24]:
revenue_pivot.reset_index(inplace=True)

Переименую столбцы:

In [26]:
revenue_pivot.columns = ['id_tt', 'апрель_2021', 'май_2021', 'июнь_2021', 'июль_2021', 'апрель_2022', 'май_2022', 'июнь_2022']

Удалю столбец июль 2021 года

In [ ]:
revenue_pivot.drop(['июль_2021'], axis=1)

Добавлю информацию о магазинах:

In [28]:
revenue = pd.merge(info_tt, revenue_pivot, how='left', on='id_tt')

Удалю строки с пропущенными значениями

In [30]:
revenue.dropna(axis=0, inplace=True)
revenue.isna().sum().sum()

0

Посчитаю среднемесячную выручку за апрель-июнь 2021 и 2022 года.

In [31]:
revenue['2021(4-6)'] = ((revenue['апрель_2021'] + revenue['май_2021'] + revenue['июнь_2021']) / 3).astype(int)

In [32]:
revenue['2022(4-6)'] = ((revenue['апрель_2022'] + revenue['май_2022'] + revenue['июнь_2022']) / 3).astype(int)

Посчитаю изменение выручки май к марту:

In [33]:
revenue['Изменение, %'] = ((revenue['2022(4-6)']  - revenue['2021(4-6)']) / revenue['2021(4-6)']*100).round(2)

Посчитаю эффективность продаж с квадратного метра:

In [34]:
revenue['revenue_m2'] = (revenue['2021(4-6)'] / revenue['ploshad']).astype(int)

Создам таблицу эффективности:

In [36]:
effectiveness_tt = revenue[['id_tt', 'ploshad', 'revenue_m2']]

Посмотрю распределение данных:

In [37]:
revenue['Изменение, %'].describe().round()

count    78.0
mean    -10.0
std      18.0
min     -41.0
25%     -22.0
50%     -11.0
75%      -1.0
max      85.0
Name: Изменение, %, dtype: float64

In [38]:
revenue['revenue_m2'].describe().round()

count        78.0
mean      81102.0
std       28949.0
min       17723.0
25%       64558.0
50%       78958.0
75%       93942.0
max      197873.0
Name: revenue_m2, dtype: float64

Список хороших ТТ:

In [39]:
GOOD_tt = revenue[revenue['Изменение, %'] >= 0]

Выведу в Excel:

In [40]:
#GOOD_tt.to_excel('GOOD_tt.xlsx')

In [41]:
GOOD_tt = GOOD_tt['id_tt']

Список плохих магазинов:

In [43]:
BAD_tt = revenue[revenue['Изменение, %'] < 0]

Выведу в Excel:

In [44]:
#BAD_tt.to_excel('BAD_tt.xlsx')

In [45]:
BAD_tt = BAD_tt['id_tt']

## Геолокация ТТ

In [47]:
#lat = revenue['Shirota']
#lon = revenue['Dolgota']
#name_1 = revenue['id_tt']
#name_2 = revenue['Изменение, %']
#name_3 = revenue['revenue_m2']

In [48]:
#Map = folium.Map(location=[55.73444,36.88808], zoom_start = 10)

In [49]:
#def color_change(name_2):
    #if(name_2 > 0):
        #return('green')
    #elif(0 >= name_2 > (-5)):
        #return('orange')
    #else:
        #return('red')

In [50]:
#for lat_1, lon_1, name_1, name_2, name_3 in zip(lat_1, lon_1, name_1, name_2, name_3):
    #folium.CircleMarker(location=[lat_1, lon_1], radius = 10, popup=("№" + str(name_1) + "  " + str(name_2) + "%" + "  " + str(name_3)), fill_color=color_change(name_2), color="gray", fill_opacity = 0.8).add_to(map)

In [51]:
#map.save("map1.html")
#map

In [52]:
#def color_change(icon):
    #if icon > 110000:
        #return 'green'
    #elif 110000 >= icon > 50000:
        #return 'orange'
    #else:
        #return 'red'

In [53]:
#for lat, lon, name_1, name_2, name_3 in zip(lat, lon, name_1, name_2, name_3):
    #folium.CircleMarker(location=[lat, lon], radius = 10, popup="№" + str(name_1) + "  " + str(name_2) + "%" + "  " + str(name_3), fill_color=color_change(name_3), color="gray", fill_opacity = 0.8).add_to(map)

In [54]:
#map.save("map2.html")
#map

## Качественный анализ:

### Товарная номенклатура за май 2021 и 2022 годов по всем ТТ:

Подготовлю таблицы:

In [55]:
tt_may21 = pd.merge(may21, sku, how='left', on='id_tov')
tt_may22 = pd.merge(may22, sku, how='left', on='id_tov')

Удалю пакеты:

In [56]:
tt_may21 = tt_may21[~(tt_may21['id_tov'] == 647)]
tt_may22 = tt_may22[~(tt_may22['id_tov'] == 647)]

__Май 2021 года:__

Товарная номенклатуры по объёму продаж:

In [57]:
tt_may21_gr2 = tt_may21.groupby('Name_gr2').agg(may21_revenue=('BaseSum', 'sum')
                                                ).reset_index()

In [58]:
tt_may21_gr2['may21_revenue'] = tt_may21_gr2['may21_revenue'].astype(int)

Добавлю столбец доли товарной номенклатуры в общем объёме продаж:

In [59]:
tt_may21_gr2['may21_percent, %'] = (tt_may21_gr2['may21_revenue'] / tt_may21_gr2['may21_revenue'].sum() * 100).round(2)

In [ ]:
tt_may21_gr2.sort_values(by='may21_percent, %', ascending=False).head(15)

Выведу в Excel:

In [61]:
#tt_may21_gr2.sort_values(by='may21_percent, %', ascending=False).head(15).to_excel('tt_may21_gr2.xlsx')

__Май 2022 года__

Товарная номенклатура по объёму продаж:

In [62]:
tt_may22_gr2 = tt_may22.groupby('Name_gr2').agg(may22_revenue=('BaseSum', 'sum')).reset_index()

In [63]:
tt_may22_gr2['may22_revenue'] = tt_may22_gr2['may22_revenue'].astype(int)

Добавлю столбец доли товарной номенклатуры в общем объёме продаж:

In [ ]:
tt_may22_gr2['may22_percent, %'] = (tt_may22_gr2['may22_revenue'] / tt_may22_gr2['may22_revenue'].sum() * 100).round(2)
tt_may22_gr2.sort_values(by='may22_percent, %', ascending=False).head(15)

Выведу в Excel:

In [65]:
#tt_may22_gr2.sort_values(by='may22_percent, %', ascending=False).head(15).to_excel('tt_may22_gr2.xlsx')

#### Вывод

Заметно, что за год предпочтения покупателей сдвинулись в сторону уменьшения покупок овощей, зелени и фруктов.

## Исследование хороших и плохих ТТ.

### Хорошие ТТ:

In [66]:
tt_GOOD_may22 = pd.merge(GOOD_tt, tt_may22, how='left', on='id_tt')

Товарная номенклатура по объёму продаж:

In [67]:
tt_GOOD_may22_gr2 = tt_GOOD_may22.groupby('Name_gr2').agg(may22_revenue_good=('BaseSum', 'sum')
                                                ).reset_index()

In [68]:
tt_GOOD_may22_gr2['may22_revenue_good'] = tt_GOOD_may22_gr2['may22_revenue_good'].astype(int)

Добавлю столбец доли товарной номенклатуры в общем объёме продаж:

In [69]:
tt_GOOD_may22_gr2['may22_percent_good'] = (tt_GOOD_may22_gr2['may22_revenue_good'] / tt_GOOD_may22_gr2['may22_revenue_good'].sum() * 100).round(4)

Товарная группа по объёму продаж:

In [71]:
tt_GOOD_may22_gr = tt_GOOD_may22.groupby('Name_gr').agg(may22_revenue_good=('BaseSum', 'sum')
                                                ).reset_index()

In [72]:
tt_GOOD_may22_gr['may22_revenue_good'] = tt_GOOD_may22_gr['may22_revenue_good'].astype(int)

Добавлю столбец доли товарной группы в общем объёме продаж:

In [73]:
tt_GOOD_may22_gr['may22_percent_good'] = (tt_GOOD_may22_gr['may22_revenue_good'] / tt_GOOD_may22_gr['may22_revenue_good'].sum() * 100).round(4)

### Плохие ТТ

Подготовлю таблицы

In [75]:
tt_BAD_may22 = pd.merge(BAD_tt, tt_may22, how='left', on='id_tt')

Товарная номенклатура по объёму продаж:

In [76]:
tt_BAD_may22_gr2 = tt_BAD_may22.groupby('Name_gr2').agg(may22_revenue_bad=('BaseSum', 'sum')
                                                ).reset_index()

In [77]:
tt_BAD_may22_gr2['may22_revenue_bad'] = tt_BAD_may22_gr2['may22_revenue_bad'].astype(int)

Добавлю столбец доли товарной номенклатуры в общем объёме продаж:

In [78]:
tt_BAD_may22_gr2['may22_percent_bad'] = (tt_BAD_may22_gr2['may22_revenue_bad'] / tt_BAD_may22_gr2['may22_revenue_bad'].sum() * 100).round(4)

Товарная группа по объёму продаж:

In [80]:
tt_BAD_may22_gr = tt_BAD_may22.groupby('Name_gr').agg(may22_revenue_bad=('BaseSum', 'sum')
                                                ).reset_index()

In [81]:
tt_BAD_may22_gr['may22_revenue_bad'] = tt_BAD_may22_gr['may22_revenue_bad'].astype(int)

Добавлю столбец доли товарной группы в общем объёме продаж:

In [82]:
tt_BAD_may22_gr['may22_percent_bad'] = (tt_BAD_may22_gr['may22_revenue_bad'] / tt_BAD_may22_gr['may22_revenue_bad'].sum() * 100).round(4)

### Сравнительный анализ по товарной номенклатуре за май 2022 года.

Создам таблицу для сравнительного анализа:

In [84]:
compar_analysis = pd.merge(tt_BAD_may22_gr2, tt_GOOD_may22_gr2, how='outer', on='Name_gr2')

In [85]:
compar_analysis.isna().sum().sum()

58

Заполню пропуски нулями:

In [86]:
compar_analysis = compar_analysis.fillna(0)

In [87]:
compar_analysis.isna().sum().sum()

0

Удалю ненужные столбцы:

In [88]:
#compar_analysis = compar_analysis.drop(columns =['may22_percent_bad', 'may22_percent_good'])

Выделю товарную группу с продажами меньше 1000 рублей в плохих магазинах и продажами больше 1000 рублей в хороших:

In [ ]:
gr2_bad = compar_analysis.loc[(compar_analysis['may22_revenue_bad'] < 1000) & (compar_analysis['may22_revenue_good'] > 1000)]
gr2_bad

Удалю товарную номенклатуру с продажами меньше 10000 рублей

In [90]:
compar_analysis = compar_analysis[(compar_analysis['may22_revenue_bad'] > 10000) & (compar_analysis['may22_revenue_good'] > 10000)]

Добавлю столбец ранга в зависимости от выручки ТТ:

In [91]:
compar_analysis['rank_bad'] = compar_analysis['may22_revenue_bad'].rank(ascending = False).astype('int')
compar_analysis['rank_good'] = compar_analysis['may22_revenue_good'].rank(ascending = False).astype('int')

In [92]:
compar_analysis['dif_rank'] = compar_analysis['rank_good'] - compar_analysis['rank_bad']

Выведу таблицу compar_analysis в Excel:

In [94]:
#compar_analysis.to_excel('compar_analysis.xlsx')

## Выбор плохих ТТ для эксперимента:

Подготовлю таблицу:

In [96]:
experiment_tt = pd.merge(BAD_tt, effectiveness_tt, how='left', on='id_tt')

Выберу ТТ из расчета, чтобы минимум и максимум интервала не отклонялись более 15% от среднего, для чего задам площадь ТТ в интервале 110-140 и эффективность в интервале 50000 - 70000

In [98]:
experiment_tt = experiment_tt[(experiment_tt['ploshad'].between(110, 160)) & (experiment_tt['revenue_m2'].between(50000, 70000))]

По этим условиям нашлось 8 ТТ

Добавлю информацию по ТТ:

In [100]:
experiment_tt = experiment_tt['id_tt']

In [101]:
experiment_tt = pd.merge(experiment_tt, info_tt, how='left', on='id_tt')

Выведу в excel:

In [103]:
#experiment_tt.to_excel('experiment_tt.xlsx')